### Installation

In [ ]:
#!pip install "sagemaker>=2.48.0" --upgrade
#!pip install sagemaker gdown

### Dataset download

In [ ]:
!gdown "https://drive.google.com/u/0/uc?export=download&confirm=2rTA&id=130flJ0u_5Ox5D-pQFa5lGiBLqILDBmXX" 

In [ ]:
json_path="evaluate_news.json"
target_path='dataset.jsonl'

import json
with open(json_path,"r+") as f, open(target_path, 'w') as outfile:
    JSON_file = json.load(f)
    for entry in JSON_file:
        json.dump(entry, outfile)
        outfile.write('\n')


### Permissions

In [ ]:
import sagemaker

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

### Preprocessing


The preproccsing will be done in the train.py when executing the script.

### Fine-tuning & starting Sagemaker Training Job

In [ ]:
from sagemaker.huggingface import HuggingFace

role = sagemaker.get_execution_role()
hyperparameters = {
	'model_name': 'facebook/bart-large-cnn',
	'dataset_name': 'dataset.jsonl',
	'num_epochs': 10,
	'train_batch_size': 8,
	'eval_batch_size': 8,
	'learning_rate': 5e-5,
	'weight_decay_rate': 0.01,
	'num_warmup_steps': 0,
	'fp16': True
}


# creates Hugging Face estimator
huggingface_estimator = HuggingFace(
	entry_point='train.py',
	source_dir='./scripts',
	instance_type='ml.p3.2xlarge',
	instance_count=1,
	role=role,
	transformers_version='4.12.3',
	tensorflow_version='2.5.1',
	py_version='py37',
	hyperparameters = hyperparameters
)


### Uploading dataset to s3

In [ ]:
from sagemaker.s3 import S3Uploader

dataset_uri = S3Uploader.upload(local_path="dataset.jsonl",desired_s3_uri=f"s3://{sess.default_bucket()}/dataset/dataset.jsonl")

### Start training

In [ ]:
huggingface_estimator.fit({"dataset": dataset_uri})

### Deploying the endpoint

In [ ]:
predictor = huggingface_estimator.deploy(1,"ml.g4dn.xlarge")

In [ ]:
text_input= {"inputs":"Financial technology giant Revolut reported its first-ever annual profit in 2021, according to financial accounts released Wednesday, as subscriptions to its paid packages and overall usage of its app grew sharply.The company reported revenues of £636.2 million ($767.1 million) for the year, three times what it made the previous year, and swung to a pre-tax profit of £59.1 million. In 2020, Revolut recorded a pre-tax loss of £205 million. Mikko Salovaara, Revolut’s chief financial officer, told CNBC the results were the product of Revolut’s diversified business and diligent cost control.The worst possible scenario would be if Revolut wasn’t sustainable or if it were to require external funding,” Salovaara said. “The reality is we don’t require external funding. We continue to invest in our business providing products people can rely on.”For 2022, Revolut gave a trading update saying it expects revenues to have grown more than 30% to £850 million. As a privately held firm, it is not required to share frequent quarterly reports.Revolut’s announcement is a rare positive piece of news in a fintech market that has been plagued by mass layoffs and huge valuation cuts as investors reassess the space amid worsening macroeconomic conditions.Klarna, the Swedish buy now, pay later fintech, saw its valuation plunge 85% to $6.7 billion last year. On Tuesday, the firm posted a record $1 billion loss in its 2022 fiscal year.Asked about Revolut’s valuation Wednesday, Salovaara said he couldn’t say how much the firm was worth since it hasn’t raised cash since 2021, but he’d be “hard pressed to believe investors wouldn’t continue to be pleased with our performance.”However, Revolut was late to producing its accounts to the U.K. company register, Companies House, in time for a Dec. 31 deadline. They were finally signed off by BDO, Revolut’s auditors, last month.Revolut reportedly faced concerns from U.K. regulators over the robustness of its internal financial controls. In September, BDO’s audit of Revolut’s 2021 accounts was deemed “inadequate” by the Financial Reporting Council, which said that “the risk of an undetected material misstatement was unacceptably high.”The company, which has no physical branches, offers digital banking, money transfers, and cryptocurrency and stock trading through a single app. It competes with the likes of Wise, Monzo and Starling.Founded in 2015 by former Lehman Brothers trader Nikolay Storonsky and software developer Vlad Yatsenko, Revolut has quickly grown to become one of Europe’s largest fintech unicorns, with a valuation of $33 billion.Revolut has been pushing hard into overseas markets, in particular the U.S., where it currently has over 500,000 clients. The firm has also opened operations in Brazil, Mexico and India. In November, Revolut announced it has 25 million users worldwide.Closer to home, though, the company’s growth plans have been dealt some setbacks. Revolut has been pursuing a banking license in the U.K. for the past two years, in an effort to source more of its income from lending activity.That process has been a drawn out one, and it is believed the wait is in connection with the delays to the publication of Revolut’s results. Revolut has also faced criticisms over an aggressive working culture, which has reportedly led to departures of key regulatory and compliance executives.Revolut hopes to obtain its U.K. banking license “very soon,” Salovaara said. Pressed on when the firm would eventually secure its license, he suggested it was likely to happen before the year is out.While Revolut’s full-year 2022 results are yet to be disclosed, one thing is clear — the firm’s crypto business deteriorated sharply. Salovaara said that in 2021, crypto accounted for roughly a third of sales, but in 2022 this dropped to between 5% to 10%."}

predictor.predict(text_input)

In [ ]:
predictor.delete_endpoint()